[< __INTRO MODULE 5__](./0.Introduction.ipynb)

---

# Index:

- [Introduction to logging](#introduction-to-logging)
- [Loggers](#loggers)
    - [Logging levels](#logging-levels)
    - [Setting a level](#setting-a-level)
    - [Adding format](#adding-format)
    - [Saving logs](#saving-logs)
- [Basic configuration](#basic-configuration)

---

# Introduction to logging

Python has a system for logging everything that happens in the script. Do not confuse a log with the use of `print()`__ to terminally display an error that has been generated.

Logs are used to detect when an error is occurring in the system at a point in time.

An __example of LOG__ would be the actions performed by a user on your web platform. A priori these logs would not be interesting, however, __if we detect a posteriori that on day X something strange happened in a user's account we will have the LOGS__ to be able to get an idea of what happened.

Logs can be displayed in different formats:
- A file
- Output stream
- Load them into an external system

Python offers all this functionality through the `logging` module.

---

# Loggers

The system with which Logs can be generated is called a logger and they work like classes in python, i.e. there is a hierarchy topped by a logger called `root`.

Then all its descendants inherit features from it, such as:
- [Logging-levels](#logging-levels): How severe messages must be, as a minimum, to be processed.
- [Handlers](#saving-logs): How messages will be displayed
- [Format](#adding-format): The format in which the messages will be displayed.
- Filters: Message filters can be applied so that they are not displayed by the current logger and children.
- Propagation: Allow generated logs to propagate upwards (enabled by default)

If the python script is simple, it is recommended to use the `root` logger. We can access it by creating the logger object through `getLogger()` without sending any parameters.

Here is an example:

In [4]:
import logging

logger = logging.getLogger()
logger.critical("Hi from root logger")

Hi from root logger


For more complex structures, where inheritance-linked loggers are required, specify in the `getLogger()` parameter the desired hierarchy using dot-separated names.

For example, in the following we will generate the logger father, child1 and child2. Obviously, the child loggers are inheritors of father:

In [ ]:
father = logging.getLogger("father")
child1 = logging.getLogger("father.child1")
child2 = logging.getLogger("father.child2")

As we can see, the creation of loggers is very similar to modules in python, for example, let's imagine that we want to import data from child2, surely what we could do is a ... `import father.child2`.

The nomenclature of the loggers is analogous to the import of modules in Python, which facilitates the organisation and management of logs in large applications. Therefore, it is advisable to use the __name__ parameter in getLogger to automatically reflect the module in use, thus improving log management and tracking.

In other words, what we should do as a general rule is:

In [ ]:
logger = logging.getLogger(__name__)


---

## Logging levels

As we have seen in the previous example (`logger.critical(‘Hi from root logger’)`), for the logger to generate the message we have to explicitly indicate the message.

We have different levels of criticality when generating it, these are:

In [5]:
# Creating the logger
logging.basicConfig()
logger = logging.getLogger()

# Messages
logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

CRITICAL:root:Your CRITICAL message
ERROR:root:Your ERROR message


Of the 5 messages, only 3 have been generated. This is due to the configuration of the logger itself, which defines the minimum level of criticality for a message to be displayed. 

> __NOTE__: We will go into detail about the initial log message (`...:root:...`) in the section "[Adding format](#adding-format)".

As we can guess, `root`, only shows up to warnings, which may lead us to ask __What should I do to show the `info` and `debut` message logs?__?

To define the level of criticality defined, we first need to know the existing levels

| Level name | Value |
|--|--|
| CRITICAL | 50 |
| ERROR | 40 |
| WARNING | 30 |
| INFO | 20 |
| DEBUG | 10 |
| NOTSET | 0 |

<br>

> __NOTE__: Please note that Logger allows you to create your own levels although the existing ones from the module itself should be enough.

Then, in order to define the level of messages that will be shown with the generated logger, we must use the `setLevel()` method, indicating as a parameter the minimum level we want to show.

If we query the table we can use the `DEBUT` or `NOTSET` value to display the `INFO` and `DEBUG` messages.

Here is an example:

In [6]:
logger.setLevel(logging.NOTSET)

# Messages
logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

CRITICAL:root:Your CRITICAL message
ERROR:root:Your ERROR message
INFO:root:Your INFO message
DEBUG:root:Your DEBUG message


Regarding the criticality inheritance, it is important to note that when creating a new logger (not root) it will have a `NOTSET` criticality level, however, instead of displaying all messages in a predefined way, it will inherit the criticality of its upper layer.

Practical Example:

Imagine you have the following logger scheme:

>Root Logger (`WARNING` level).<br>
>&nbsp;&nbsp;&nbsp;&nbsp;└── Logger A (level `NOTSET`).<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Logger B (level `NOTSET`)

- Logger A inherits the `WARNING` level from the root logger because its level is `NOTSET`. This means that Logger A will not process DEBUG or INFO messages.
- Logger B also inherits the `WARNING` level because both its level and that of its parent (Logger A) are `NOTSET`.

However, if we set the root logger to have a `NOTSET` criticality level, all its children will display all messages by default. Unless between the child and root there is a logger with a defined criticality level. Then that child will inherit that criticality level.

---

## Saving logs

As mentioned [above](#loggers), a logger can define where the generated logs will be stored, be it on the terminal, in a file or on an external system.

To serve this purpose, `logging` introduces `Handlers`, in particular, and for saving logs to files, it introduces the `FileHandler`.

To generate a handler to save logs to a file, do the following:
1. Instantiate a `FileHandler`.
2. When instantiating, the following arguments are provided:
    - Name of the file
    - Mode to work with (the same mode used in the `open()` function).
3. On the instantiated handler we define the level at which these messages will be shown with the `setLevel` method.
4. Bind the Handler in the Logger through the `addHandler` method.

> NOTE: Note that each handler can have its own criticality level. This opens the door that, for the same logger, the critical messages are stored in a file and everything lower is only displayed on screen.

An example is shown below:

In [2]:
logger = logging.getLogger(__name__)

handler = logging.FileHandler('./persistance/example_logging_handler_module_4', mode='w')
handler.setLevel(logging.CRITICAL)

logger.addHandler(handler)

logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

If we look at the [generated file](./persistance/example_logging_handler_module_4), we will see that it has no formatting whatsoever, i.e. it is the literal message received.

However... this message can be formatted a bit in order to provide all the necessary context, this is done through the [formatters](#adding-format).

---

## Adding format

To apply a format we only need to include in the handler the format we want to apply.

This format is generated through the `LogRecord` object which has [many attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes) to format the generated message, here is a selection of options:
- `%(name)s`: This pattern will be replaced by the name of the logger that calls the logging method. In our case, it's the root logger;
- `%(levelname)s`: This pattern will be replaced with the set login level. In our case, this is the CRITICAL level;
- `%(asctime)s`: This pattern will be replaced with a human-readable date format that indicates when the LogRecord object was created. The decimal value is expressed in milliseconds;
- `%(message)s`: This pattern will be replaced by the defined message. In our case, it's 'Your CRITICAL message'.

An example is shown below:

In [3]:
import logging

FORMAT = '%(name)s:%(levelname)s:%(asctime)s:%(message)s'

logger = logging.getLogger(__name__)

handler = logging.FileHandler('./persistance/example_logging_handler_formatted_module_4', mode='w')
handler.setLevel(logging.CRITICAL)

formatter = logging.Formatter(FORMAT)
handler.setFormatter(formatter)

logger.addHandler(handler)

logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')


---

## Basic configuration

The logging module has a system for defining the configuration of the root logger and its children. This system can be accessed through the `basicConfig()` method.

When calling it we can indicate in the parameters how each of the logger's features will be configured, however, if we don't inform anything a predefined configuration will be set (as we have seen in [examples](#logging-levels) above).

The default configuration (with no arguments) does the following:
1. Generates a `StreamHandler` to display the log output on screen.
2. The `StreamHandler` is created from the default `Formatter`.
    - The format consists of `LEVEL:LOGGER_NAME:MESSAGE`.
3. The created andler is passed to the root logger.

If instead we send the arguments to `basicConfig` we can tell it the criticality level, output system and how the log file is treated.

> __NOTE__: Keep in mind that `basicConfig` can only be run once, so second runs will have no effect on the logger. This is especially important for this demo as we will need to (manually) restart the Jupyter Notebook kernel for it to work.

An example is shown below:

In [1]:
import logging

FORMAT = '%(name)s:%(levelname)s:%(asctime)s:%(message)s'

logging.basicConfig(level=logging.CRITICAL, filename='./persistance/example_logging_module_4_prod.log', filemode='a', format=FORMAT)

loggerNew = logging.getLogger()

loggerNew.critical('Your CRITICAL message')
loggerNew.error('Your ERROR message')
loggerNew.warning('Your WARNING message')
loggerNew.info('Your INFO message')
loggerNew.debug('Your DEBUG message')

As we can see [here](./persistance/example_logging_module_4_prod.log), only the message with the critical level has been recorded in the log.

The following are some of the arguments that `basicConfig` may receive:
- `level`: The level that is set as the minimum level for displaying messages in root.
- `filename`: What the generated log file will be called (this will imply that logs will not be displayed per terminal).
- `filemode`: How the log file will be treated (this is related to the `open()` mode of files)
- `format`: Format in which the messages will be displayed
    - The default format is: `%(levelname)s:%(name)s:%(message)s`.


---

[< __INTRO MODULE 5__](./Introduction.ipynb)